1. **Installing required library and packages and setting Groq API:**

In [2]:
pip install groq

In [3]:
from google.colab import userdata
api_key = userdata.get("GROQ_API_KEY")

In [4]:
import groq

client = groq.Client(api_key=api_key)

In [5]:
!pip install langchain

2. Downloading Enron dataset for email

In [6]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("wcukierski/enron-email-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/wcukierski/enron-email-dataset/versions/2


In [7]:
import os

In [8]:
import pandas as pd

# Path to the dataset
dataset_path = "/root/.cache/kagglehub/datasets/wcukierski/enron-email-dataset/versions/2/emails.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(dataset_path)

# Display the first few rows of the DataFrame
print(df.head())

                       file                                            message
0     allen-p/_sent_mail/1.  Message-ID: <18782981.1075855378110.JavaMail.e...
1    allen-p/_sent_mail/10.  Message-ID: <15464986.1075855378456.JavaMail.e...
2   allen-p/_sent_mail/100.  Message-ID: <24216240.1075855687451.JavaMail.e...
3  allen-p/_sent_mail/1000.  Message-ID: <13505866.1075863688222.JavaMail.e...
4  allen-p/_sent_mail/1001.  Message-ID: <30922949.1075863688243.JavaMail.e...


In [9]:
# Extract email content from the 'message' column
emails = df['message'].tolist()

# Display the first few emails
for i, email in enumerate(emails[:5]):
    print(f"Email {i+1}:")
    print(email)
    print("\n---\n")

Email 1:
Message-ID: <18782981.1075855378110.JavaMail.evans@thyme>
Date: Mon, 14 May 2001 16:39:00 -0700 (PDT)
From: phillip.allen@enron.com
To: tim.belden@enron.com
Subject: 
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: Tim Belden <Tim Belden/Enron@EnronXGate>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Here is our forecast

 

---

Email 2:
Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged

**Extracting Email body **

In [10]:
def extract_email_body(email):
    # Split the email into headers and body
    parts = email.split("\n\n", 1)  # Split on the first empty line
    if len(parts) > 1:
        return parts[1]  # Return the body
    return email  # If no body is found, return the entire email

# Extract email bodies
email_bodies = [extract_email_body(email) for email in emails]

# Display the first few email bodies
for i, body in enumerate(email_bodies[:5]):
    print(f"Email Body {i+1}:")
    print(body)
    print("\n---\n")

Email Body 1:
Here is our forecast

 

---

Email Body 2:
Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the presenter speaks and the others are quiet just waiting for their turn.   The meetings might be better if held in a round table discussion format.  

My suggestion for where to go is Austin.  Play golf and rent a ski boat and jet ski's.  Flying somewhere takes too much time.


---

Email Body 3:
test successful.  way to go!!!

---

Email Body 4:
Randy,

 Can you send me a schedule of the salary and level of everyone in the 
sche

Truncating long emails to avoid the API payload limits.

In [12]:
def truncate_email(email, max_length=500):
    """Truncate the email to a maximum length."""
    return email[:max_length]

truncated_email_bodies = [truncate_email(email) for email in email_bodies]

for i, email in enumerate(truncated_email_bodies[:5]):
    print(f"Truncated Email {i+1}:")
    print(email)
    print("\n---\n")

Truncated Email 1:
Here is our forecast

 

---

Truncated Email 2:
Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too

---

Truncated Email 3:
test successful.  way to go!!!

---

Truncated Email 4:
Randy,

 Can you send me a schedule of the salary and level of everyone in the 
scheduling group.  Plus your thoughts on any changes that need to be made.  
(Patti S for example)

Phillip

---

Truncated Email 5:
Let's shoot for Tuesday at 11:45.  

---



**Calssifying Email into Work, Personal, Promotional, and Spam**

In [13]:
# Classify an email using Groq API
def classify_email(email_content):
  completion = client.chat.completions.create(
      model="qwen-2.5-32b",
      messages=[
          {
              "role": "system",
              "content": "your role is to categorize email into work, personal, promotional, spam just categorize the email don't give justification"
          },
          {
              "role": "user",
              "content": email_content
          },
      ],
      temperature=0.6,
      max_completion_tokens=4096,
      top_p=0.95,
      stream=False,
      stop=None,
  )
  return completion.choices[0].message.content

# Classify emails
categorized_emails = []
for email in truncated_email_bodies[:20]:  # Process the first 10 emails
    category = classify_email(email)
    categorized_emails.append({"email": email, "category": category})

# Display categorized emails
print("Categorized Emails:")
for email in categorized_emails:
    print(f"Email: {email['email'][:100]}...")  # Display first 100 characters
    print(f"Category: {email['category']}")
    print()



Categorized Emails:
Email: Here is our forecast

 ...
Category: Work

Email: Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepa...
Category: Work

Email: test successful.  way to go!!!...
Category: personal

Email: Randy,

 Can you send me a schedule of the salary and level of everyone in the 
scheduling group.  P...
Category: Work

Email: Let's shoot for Tuesday at 11:45.  ...
Category: work

Email: Greg,

 How about either next Tuesday or Thursday?

Phillip...
Category: work

Email: Please cc the following distribution list with updates:

Phillip Allen (pallen@enron.com)
Mike Grigs...
Category: Work

Email: any morning between 10 and 11:30...
Category: work

Email: 1. login:  pallen pw: ke9davis

 I don't think these are required by the ISP 

  2.  static IP addre...
Category: Work

Email: ---------------------- Forwarded by Phillip K Allen/HOU/ECT on 10/16/2000 
01:42 PM ----------------...
Category: Work

Email: Mr. Buckner,

 For d

**Summarizing Email into 1 to 2 sentences**

In [14]:
def summarize_email(email_content):
  completion = client.chat.completions.create(
      model="qwen-2.5-32b",
      messages=[
          {
              "role": "system",
              "content": "Summarize this email in one or two sentences."
          },
          {
              "role": "user",
              "content": email_content
          },
      ],
      temperature=0.6,
      max_completion_tokens=4096,
      top_p=0.95,
      stream=False,
      stop=None,
  )
  return completion.choices[0].message.content

# Summarize emails
summarized_emails = []
for email in email_bodies[:5]:  # Process the first 5 emails for demonstration
    summary = summarize_email(email)
    summarized_emails.append({"email": email, "summary": summary})

# Display summarized emails
for email in summarized_emails:
    print(f"Email: {email['email'][:100]}...")  # Display first 100 characters
    print(f"Summary: {email['summary']}")
    print()


Email: Here is our forecast

 ...
Summary: The email contains a forecast, but it lacks specific details, making it impossible to summarize without additional information.

Email: Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepa...
Summary: The email suggests separating business meetings from leisure trips and recommends a more interactive round table format for meetings. It also proposes going to Austin for recreational activities like golf and water sports, preferring it over flying due to time efficiency.

Email: test successful.  way to go!!!...
Summary: The test was successful, and congratulations are in order.

Email: Randy,

 Can you send me a schedule of the salary and level of everyone in the 
scheduling group.  P...
Summary: Phillip has asked Randy to provide a schedule detailing the salary and level of everyone in the scheduling group, along with Randy's recommendations for any necessary changes.

Email: Let's shoot for Tues

**Natural language queries using the Groq API you can chat with your email**

In [25]:
def process_query(user_query, emails):
    """
    Process a natural language query and retrieve relevant emails using the Groq API.

    Args:
        user_query (str): The user's query (e.g., "Show me emails about project updates").
        emails (list): A list of email bodies to search through.

    Returns:
        list: A list of relevant emails based on the query.
    """
    try:
        # Combine the user query and email content into a single prompt
        prompt = (
            f"Retrieve emails related to the following query: '{user_query}'. "
            f"Here are the emails:\n\n" + "\n\n".join(emails) + "\n\n"
            "Return only the relevant emails without any additional explanation. "
            "If no emails are relevant, return 'No relevant emails found.'"
        )
        completion = client.chat.completions.create(
            model="qwen-2.5-32b",
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI assistant that retrieves relevant emails based on a user query. "
                               "Return only the relevant emails without any additional explanation."
                },
                {
                    "role": "user",
                    "content": prompt
                },
            ],
            temperature=0.3,
            max_tokens=4096,  # Use max_tokens instead of max_completion_tokens
            top_p=0.95,
            stream=False,
            stop=None,
        )

        # Extract the response from the Groq API
        response = completion.choices[0].message.content

        # Split the response into individual emails (assuming the API returns one email per line)
        relevant_emails = response.strip().split("\n")

        # Filter out empty strings (if any)
        relevant_emails = [email for email in relevant_emails if email.strip()]

        # If no relevant emails are found, return a default message
        if not relevant_emails:
            return ["No relevant emails found."]

        return relevant_emails

    except Exception as e:
        print(f"Error processing query: {e}")
        return ["An error occurred while processing the query."]

# Process a query
user_query = input("Enter a query: ")
relevant_emails = process_query(user_query, truncated_email_bodies[:50])  # Process the first 50 emails to avoid payload

# Display relevant emails
print(f"Query: {user_query}")
if relevant_emails:
    for email in relevant_emails:
        print(f"Relevant Email: {email[:100]}...")
    print("No relevant emails found.")

Enter a query: show the schedule?
Query: show the schedule?
Relevant Email: Randy,...
Relevant Email:  Can you send me a schedule of the salary and level of everyone in the ...
Relevant Email: scheduling group.  Plus your thoughts on any changes that need to be made.  ...
Relevant Email: (Patti S for example)...
Relevant Email: Phillip...
Relevant Email: ---------------------- Forwarded by Phillip K Allen/HOU/ECT on 09/26/2000 ...
Relevant Email: 12:07 PM ---------------------------...
Relevant Email: 	From:  Cindy Cicchetti                           09/26/2000 09:23 AM...
Relevant Email: To: Phillip K Allen/HOU/ECT@ECT, Hunter S Shively/HOU/ECT@ECT, Scott ...
Relevant Email: Mills/HOU/ECT@ECT, Allan Severude/HOU/ECT@ECT, Russ Severson/HOU/ECT@ECT, ...
Relevant Email: Fletcher J Sturm/HOU/ECT@ECT, Scott Neal/HOU/ECT@ECT...
Relevant Email: cc:  ...
Relevant Email: Subject: Gas Physical/Financial Position...
Relevant Email: I have scheduled and entered on each of your calendars a meet...

In [24]:
def process_query(user_query, emails):
    """
    Process a natural language query and retrieve relevant emails using the Groq API.
    """
    try:
        # Ensure emails are formatted properly
        formatted_emails = "\n\n".join([f"Email {i+1}: {email}" for i, email in enumerate(emails)])

        # More structured prompt
        prompt = (
            f"I have the following emails:\n\n{formatted_emails}\n\n"
            f"Retrieve only the emails that are directly relevant to the query: '{user_query}'.\n"
            f"Return the most relevant 5 emails. If no relevant emails exist, just return 'No relevant emails found'."
        )

        completion = client.chat.completions.create(
            model="qwen-2.5-32b",
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI email assistant. Your task is to find and return ONLY the most relevant emails to the user's query."
                },
                {
                    "role": "user",
                    "content": prompt
                },
            ],
            temperature=0.3,  # Lower temperature for more focused answers
            max_tokens=2048,  # Reduce token size to avoid too many results
            top_p=0.8,        # More focused response
            stream=False,
            stop=None,
        )

        # Extract response
        response = completion.choices[0].message.content.strip()

        # Handle no relevant emails found
        if "No relevant emails found" in response:
            return ["No relevant emails found."]

        # Split response into individual emails
        relevant_emails = response.split("\n")

        return relevant_emails[:5]  # Limit results to top 5 emails

    except Exception as e:
        print(f"Error processing query: {e}")
        return ["An error occurred while processing the query."]
user_query = "What is the schedule?"
relevant_emails = process_query(user_query, truncated_email_bodies[:50])  # Limit input to 50 emails

# Display results
print(f"Query: {user_query}")
if relevant_emails:
    for email in relevant_emails:
        print(f"Relevant Email: {email[:200]}...")  # Show only first 200 characters for readability
else:
    print("No relevant emails found.")


Query: What is the schedule?
Relevant Email: No relevant emails found....
